In [1]:
from find_source import full_html_and_txt, make_catalog
import glob
import json

In [2]:
for folder in glob.glob('/reduction/karto/SMA/polaris_test/*'):
    print(folder)

/reduction/karto/SMA/polaris_test/250101_04:01:33
/reduction/karto/SMA/polaris_test/run_polaris.m
/reduction/karto/SMA/polaris_test/250101_14:31:46
/reduction/karto/SMA/polaris_test/250102_03:39:23
/reduction/karto/SMA/polaris_test/250103_05:25:18
/reduction/karto/SMA/polaris_test/250104_03:18:27
/reduction/karto/SMA/polaris_test/250105_04:05:43
/reduction/karto/SMA/polaris_test/250106_03:59:50
/reduction/karto/SMA/polaris_test/250106_13:01:37
/reduction/karto/SMA/polaris_test/250106_15:00:08
/reduction/karto/SMA/polaris_test/250106_17:38:06
/reduction/karto/SMA/polaris_test/250107_03:21:27
/reduction/karto/SMA/polaris_test/250107_11:09:13
/reduction/karto/SMA/polaris_test/250108_04:00:57
/reduction/karto/SMA/polaris_test/250109_03:31:04
/reduction/karto/SMA/polaris_test/250109_07:40:11
/reduction/karto/SMA/polaris_test/250109_15:20:50
/reduction/karto/SMA/polaris_test/250111_03:59:34
/reduction/karto/SMA/polaris_test/250112_04:00:37
/reduction/karto/SMA/polaris_test/250114_03:31:47
/r

In [3]:
full_html_and_txt('/reduction/karto/SMA/polaris_test/250112_04:00:37')